In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./../../data/pima_indian_diabetes/diabetes_cleaned.csv", index_col=0)

In [3]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,33.6,0.627,50,1
1,1,85,66,29,26.6,0.351,31,0
2,8,183,64,0,23.3,0.672,32,1
3,1,89,66,23,28.1,0.167,21,0
5,5,116,74,0,25.6,0.201,30,0
...,...,...,...,...,...,...,...,...
763,10,101,76,48,32.9,0.171,63,0
764,2,122,70,27,36.8,0.340,27,0
765,5,121,72,23,26.2,0.245,30,0
766,1,126,60,0,30.1,0.349,47,1


In [5]:
print(df.drop(['Outcome'], axis=1))

     Pregnancies  Glucose  BloodPressure  SkinThickness   BMI  \
0              6      148             72             35  33.6   
1              1       85             66             29  26.6   
2              8      183             64              0  23.3   
3              1       89             66             23  28.1   
5              5      116             74              0  25.6   
..           ...      ...            ...            ...   ...   
763           10      101             76             48  32.9   
764            2      122             70             27  36.8   
765            5      121             72             23  26.2   
766            1      126             60              0  30.1   
767            1       93             70             31  30.4   

     DiabetesPedigreeFunction  Age  
0                       0.627   50  
1                       0.351   31  
2                       0.672   32  
3                       0.167   21  
5                       0.201   30

In [6]:
import pandas as pd
import numpy as np

# 上記のデータフレームを作成
data = {
    'Pregnancies': [6, 1, 8, 1, 5],
    'Glucose': [148, 85, 183, 89, 116],
    'BloodPressure': [72, 66, 64, 66, 74],
    'SkinThickness': [35, 29, 0, 23, 0],
    'BMI': [33.6, 26.6, 23.3, 28.1, 25.6],
    'DiabetesPedigreeFunction': [0.627, 0.351, 0.672, 0.167, 0.201],
    'Age': [50, 31, 32, 21, 30]
}

df = pd.DataFrame(data)

# 各列の最大値と最小値を取得
min_values = df.min()
max_values = df.max()

# 各列のデータを一様分布からサンプリング
num_samples = len(df)
sampled_data = {}

for column in df.columns:
    # 最小値から最大値までの一様分布からサンプリング
    sampled_data[column] = np.random.uniform(min_values[column], max_values[column], num_samples)

# サンプリングされたデータフレームを作成
sampled_df = pd.DataFrame(sampled_data)

# 結果を表示
print("Original DataFrame:")
print(df)
print("\nSampled DataFrame:")
print(sampled_df)


Original DataFrame:
   Pregnancies  Glucose  BloodPressure  SkinThickness   BMI  \
0            6      148             72             35  33.6   
1            1       85             66             29  26.6   
2            8      183             64              0  23.3   
3            1       89             66             23  28.1   
4            5      116             74              0  25.6   

   DiabetesPedigreeFunction  Age  
0                     0.627   50  
1                     0.351   31  
2                     0.672   32  
3                     0.167   21  
4                     0.201   30  

Sampled DataFrame:
   Pregnancies     Glucose  BloodPressure  SkinThickness        BMI  \
0     4.860467  161.598470      64.491282      22.147704  29.181661   
1     6.915758  137.385878      67.909364      11.311390  23.884433   
2     4.347189   96.785380      72.909059      22.258346  29.198145   
3     5.512358  123.398713      65.412903      31.416325  31.403210   
4     4.157540  

In [3]:
import pandas as pd

# サンプルデータの作成
data = {
    'Pregnancies': [6, 1, 8, 1, 5],
    'Glucose': [148, 85, 183, 89, 116],
    'BloodPressure': [72, 66, 64, 66, 74],
    'SkinThickness': [35, 29, 0, 23, 0],
    'BMI': [33.6, 26.6, 23.3, 28.1, 25.6],
    'DiabetesPedigreeFunction': [0.627, 0.351, 0.672, 0.167, 0.201],
    'Age': [50, 31, 32, 21, 30]
}

df = pd.DataFrame(data)

# 特定の列名を変更する
df.rename(columns={'Pregnancies': 'NumPregnancies', 'BloodPressure': 'BP'}, inplace=True)

# 結果を表示
print(df)


   NumPregnancies  Glucose  BP  SkinThickness   BMI  DiabetesPedigreeFunction  \
0               6      148  72             35  33.6                     0.627   
1               1       85  66             29  26.6                     0.351   
2               8      183  64              0  23.3                     0.672   
3               1       89  66             23  28.1                     0.167   
4               5      116  74              0  25.6                     0.201   

   Age  
0   50  
1   31  
2   32  
3   21  
4   30  
